In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=99280fa8997ead3face6fb40b7c61cf198eb6345252e3844842a582f8d1ae380
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
import pyspark

from pyspark.sql import SparkSession
#SparkSession is now the entry point of Spark
#SparkSession can also be construed as gateway to spark libraries


#create instance of spark class

spark=SparkSession.builder.appName('housing_price_model').getOrCreate()


#create spark dataframe of input csv file

df=spark.read.csv('/content/drive/MyDrive/cruise_ship_info.csv',inferSchema=True,header=True)

df.show(10)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [5]:
#prints structure of dataframe along with datatype
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [6]:
#In our predictive model, below are the columns
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [7]:
from pyspark.ml.feature import StringIndexer

indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')

indexed=indexer.fit(df).transform(df)


#above code will convert string to numeric feature and create a new dataframe
#new dataframe contains a new feature 'cruise_cat' and can be used further
#feature cruise_cat is now vectorized and can be used to fed to model

for item in indexed.head(5):

    print(item)

    print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)




In [8]:
from pyspark.ml.linalg import Vectors

from pyspark.ml.feature import VectorAssembler
#creating vectors from features
#Apache MLlib takes input if vector form

assembler=VectorAssembler(inputCols=['Age',

 'Tonnage',

 'passengers',

 'length',

 'cabins',

 'passenger_density',

 'cruise_cat'],outputCol='features')

output=assembler.transform(indexed)

output.select('features','crew').show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [9]:
#final data consist of features and label which is crew.

final_data=output.select('features','crew')
#splitting data into train and test

train_data,test_data=final_data.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              101|
|   mean|7.638019801980204|
| stddev|3.233203989791545|
|    min|             0.59|
|    max|             13.6|
+-------+-----------------+



In [10]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                57|
|   mean| 8.070877192982456|
| stddev|3.9526118312099094|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [11]:
#import LinearRegression library

from pyspark.ml.regression import LinearRegression
#creating an object of class LinearRegression
#object takes features and label as input arguments

ship_lr=LinearRegression(featuresCol='features',labelCol='crew')
#pass train_data to train model

trained_ship_model=ship_lr.fit(train_data)
#evaluating model trained for Rsquared error

ship_results=trained_ship_model.evaluate(train_data)



print('Rsquared Error :',ship_results.r2)
#R2 value shows accuracy of model is 92%
#model accuracy is very good and can be use for predictive analysis

Rsquared Error : 0.9544813605398774


In [12]:
#testing Model on unlabeled data
#create unlabeled data from test_data
#testing model on unlabeled data

unlabeled_data=test_data.select('features')

unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|[4.0,220.0,54.0,1...|
|[5.0,86.0,21.04,9...|
|[5.0,160.0,36.34,...|
|[6.0,112.0,38.0,9...|
|[7.0,89.6,25.5,9....|
+--------------------+
only showing top 5 rows



In [13]:
predictions=trained_ship_model.transform(unlabeled_data)
predictions.show()
#below are the results of output from test data

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,220.0,54.0,1...|21.271739496668904|
|[5.0,86.0,21.04,9...| 9.114033898203381|
|[5.0,160.0,36.34,...|15.293748410828057|
|[6.0,112.0,38.0,9...|11.112622870326526|
|[7.0,89.6,25.5,9....|10.824133054440468|
|[8.0,77.499,19.5,...| 8.430096442438705|
|[8.0,91.0,22.44,9...| 9.958161654581318|
|[9.0,59.058,17.0,...| 7.340683021224405|
|[9.0,110.0,29.74,...|11.970586563083168|
|[9.0,113.0,26.74,...|11.371217092465935|
|[10.0,58.825,15.6...| 7.080310095115826|
|[10.0,68.0,10.8,7...| 6.754771632667488|
|[10.0,90.09,25.01...| 8.749834607555409|
|[10.0,105.0,27.2,...| 11.21389654880134|
|[10.0,138.0,31.14...|13.168398774919039|
|[11.0,90.09,25.01...| 8.747297495806473|
|[11.0,91.0,20.32,...| 9.227370515613016|
|[11.0,91.62700000...|  9.19180119229311|
|[11.0,110.0,29.74...|11.976769354346743|
|[11.0,138.0,31.14...|13.165861663170102|
+--------------------+------------